In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder \
    .master("local[*]")\
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

In [3]:
from pipeline_oriented_analytics import Phase
phase = Phase.predict

data_to_predict_path = f'../data/processed/{phase.name}/features'
model_path = '../model/trip_duration_min'
predicted_data_path = '../data/reporting/trip_durations'

In [7]:
from pipeline_oriented_analytics.dataframe import ParquetDataFrame
from pipeline_oriented_analytics.transformer import SaveToParquet, DropColumns
from pyspark.ml import PipelineModel

model = PipelineModel.load(model_path)

predicted_df = PipelineModel([
    model,
    DropColumns('features'),
    SaveToParquet('../data/reporting/predicted_trip_durations')
]).transform(ParquetDataFrame(data_to_predict_path, spark))

predicted_df.show()

+--------------+-------------+--------------+---------------+--------+-----+------------+-----------+----+--------------------+---------------------+-----------------+------------------+--------------------+------------------+
|dropoff_cell_6|pickup_cell_6|pickup_cell_14|dropoff_cell_14|distance|month|day_of_month|day_of_week|hour|requests_pickup_cell|requests_dropoff_cell|pickup_cell_6_idx|dropoff_cell_6_idx|            features|        prediction|
+--------------+-------------+--------------+---------------+--------+-----+------------+-----------+----+--------------------+---------------------+-----------------+------------------+--------------------+------------------+
|          89c3|         89c3|      89c259a3|       89c258f9|    2.75|    1|           4|          2|  14|                  40|                   40|              0.0|               0.0|[0.0,0.0,2.75,1.0...| 17.32501243612355|
|          89c3|         89c3|      89c258c3|       89c258bf|     1.1|    1|           4|   